In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Myntra Search URL for Purses (First Page)
myntra_url = "https://www.myntra.com/purses"

# Set Chrome Options (Headless Mode)
options = Options()
options.add_argument("--headless")  # Run in the background
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(myntra_url)

# Wait for products to load
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "product-base")))

# Extract Purse Listings
purses = []

# Loop through multiple pages
page = 1
while True:
    print(f"📄 Scraping Page {page}...")
    
    # Extract product elements
    items = driver.find_elements(By.CLASS_NAME, "product-base")
    product_links = []

    for item in items:
        try:
            product_link = item.find_element(By.TAG_NAME, "a").get_attribute("href")
            product_links.append(product_link)
        except:
            continue  # Skip if link is missing

    # Visit each product page and extract details
    for link in product_links:
        try:
            print(f"🔗 Opening Product: {link}")
            driver.get(link)
            time.sleep(3)  # Allow page to load

            product_soup = BeautifulSoup(driver.page_source, "html.parser")

            # Extract Product Details
            brand = product_soup.find("h1", class_="pdp-title").text.strip() if product_soup.find("h1", class_="pdp-title") else "N/A"
            name = product_soup.find("h1", class_="pdp-name").text.strip() if product_soup.find("h1", class_="pdp-name") else "N/A"
            discounted_price = product_soup.find("span", class_="pdp-price").text.strip() if product_soup.find("span", class_="pdp-price") else "N/A"
            original_price = product_soup.find("span", class_="pdp-mrp").text.strip() if product_soup.find("span", class_="pdp-mrp") else "N/A"

            rating = product_soup.find("div", class_="index-overallRating")
            review_count = product_soup.find("div", class_="index-ratingsCount")
            material_info = product_soup.find("div", class_="index-productDescription")
            stock_info = product_soup.find("div", class_="pdp-price-info")
            seller_info = product_soup.find("div", class_="supplier-name")

            # Assign extracted values safely
            product_rating = rating.text.strip() if rating else "N/A"
            product_reviews = review_count.text.strip() if review_count else "N/A"
            material = material_info.text.strip() if material_info else "N/A"
            availability = "In Stock" if stock_info else "Out of Stock"
            seller = seller_info.text.strip() if seller_info else "N/A"

            # Append data
            purses.append([brand, name, discounted_price, original_price, product_rating, product_reviews, material, availability, seller, link])

        except Exception as e:
            print(f"⚠️ Error scraping product {link}: {e}")

    # Check if there is a next page
    try:
        next_button = driver.find_element(By.XPATH, "//a[@rel='next']")
        next_page_link = next_button.get_attribute("href")
        if next_page_link:
            driver.get(next_page_link)
            time.sleep(3)  # Wait for new page to load
            page += 1
        else:
            break
    except:
        print("🚫 No more pages to scrape.")
        break

# Close WebDriver
driver.quit()

# Convert to DataFrame
df = pd.DataFrame(purses, columns=["Brand", "Product Name", "Discounted Price", "Original Price", "Rating", "Reviews", "Material", "Availability", "Seller", "Product URL"])

# Save to CSV
df.to_csv("myntra_purses.csv", index=False)

print("✅ Scraping Complete! Data saved to 'myntra_purses.csv'")


📄 Scraping Page 1...
🔗 Opening Product: https://www.myntra.com/clutches/irth/irth-women-solid-purse/30188704/buy
🔗 Opening Product: https://www.myntra.com/clutches/mochi/mochi-peach-coloured-textured-embellished-purse-clutch/17295648/buy
🔗 Opening Product: https://www.myntra.com/clutches/kuber+industries/kuber-industries-set-of-3-embroidered-purses/28580562/buy
🔗 Opening Product: https://www.myntra.com/clutches/kuber+industries/kuber-industries-set-of-3-embroidered-purses/28580560/buy
🔗 Opening Product: https://www.myntra.com/clutches/lino+perros/lino-perros-women-floral-embroidered-purse/24435832/buy
🔗 Opening Product: https://www.myntra.com/clutches/kuber+industries/kuber-industries-set-of-3-embroidered-purses/28580490/buy
🔗 Opening Product: https://www.myntra.com/clutches/kuber+industries/kuber-industries-set-of-3-embroidered-purses/28580488/buy
🔗 Opening Product: https://www.myntra.com/clutches/mochi/mochi-women-grey-solid-purse--/17564540/buy
🔗 Opening Product: https://www.myntra.